In [ ]:
from __future__ import print_function
from ast import literal_eval
import pickle
import numpy as np
import csv
import collections
import math
import os
import random
import tensorflow as tf
import pdb
import itertools
import time
import pandas as pd
from random import shuffle



def one_hot(elm, size):
    vec = np.zeros(size)
    try:
        vec[int(elm)] = 1
    except: 
        vec[0] =1
    return list(vec)
    

def merchant_average(merchant_list,merchant_dict):
    """
    Calculate the average of all MERCHANT domain embeddings
    """
    merchant_av = []
    for merchant in merchant_list:
        try:
            merchant_av.append(merchant_dict[merchant])
        except:
            pass
        
    return merchant_av

def get_embedding(key, id_embedding, merchant_prior):
    """
    Look up item embedding, if not present, take the average of all merchant domains
    """
    try:
        embed = id_embedding[key]
        return list(embed)
    except:
        embed = merchant_prior
        return list(embed)

def average_keywords_domains(kw_list, kw_embed):
    """
    Get the average of all keywords used for the case when no keywords exist.
    """
    domain_av = []
    for elm in kw_list:
        try:
            this_word = kw_embed[elm] 
            domain_av.append(this_word)
        except:
            pass
    if len(domain_av) == 0:
        return [0]*300
    domain_av = np.mean(np.asarray(domain_av),axis = 0)
    return list(domain_av)

def average_keywords_url(this_id, keywords, kw_dict, av_kw):
    """ 
    Return the average keywords of domain in session, if no keywords, take average of all keywords for domain
    """
    d = []
    for elm in keywords:
        try:
            word = kw_dict[elm]
            d.append(word)
        except:
            pass
    if len(d) == 0:
        try: 
            d.append(av_kw_dict[this_id])
            return d
        except:
            return [0]*300
        
    d = np.asarray(d)
    d = np.reshape(np.mean(d,axis = 0), (-1,1))
    return(list(d))


# generate one batch per on the fly per itereation
def batch_generation(sequences, embed_dict, batch_size, max_length, embed_size, KEY_WORDS, RANDOM):
    """" Take a batch and return embedding in correct shape to be fed into RNN,
          new_batch: shape for input placeholder with last domain in seq left for prediction
          target: the prediction that is compared to.
    """

    new_batch, targets, target_merchs, target_pubs, target_merch_labels, target_labels, target_indexes,future_context = [], [], [], [], [], [], [], []
    merchant_count = 0
    history, history_no = [], []
    for i in range(len(sequences)):
        this_sequence = sequences[i]
        
        history_tmp = []
        if RANDOM == True:
            rand = np.arange(len(this_sequence))
            np.random.shuffle(rand)
            this_sequence = [this_sequence[x] for x in rand]
        history_tmp = []
        history_tmp.append(this_sequence[0][0])
        for elm in range(len(this_sequence)-1):

            target_index = mapping[str(this_sequence[elm+1][0])]
            target = np.zeros((len(ids),1)) # create a zero vector for one hot encoding
            target_labels.append(float(target_index))
            target[target_index] = 1 # fill in index with one
            targets = targets+[target] # apend to the targets list
            if this_sequence[elm+1][-1] == 1:
                target_merchs.append(merchant_count)
            else: target_pubs.append(merchant_count)


            if this_sequence[elm+1][0] in history_tmp:
                history.append(merchant_count)
            else: history_no.append(merchant_count)
            history_tmp.append(this_sequence[elm+1][0])
            merchant_count+=1

        # get the indeces for the lengths: many to many
        target_lens = np.arange(len(this_sequence)-1) + i*max_length
        target_indexes.extend(target_lens)
            
        future_context = future_context+[average_keywords_url(str(x[0]), x[1], kw, av_kw_domain_embed)+ one_hot(x[2],size_dt) + one_hot(x[3],size_hour) + one_hot(x[4], size_day) + one_hot(x[5],size_id) for x in this_sequence[1:]]
        if KEY_WORDS == True:
            new_batch = new_batch+[get_embedding([str(x[0])], embed_dict, prior)  + average_keywords_url(str(x[0]), x[1], kw, av_kw_domain_embed)for x in this_sequence[:-1]]
    
            
        else: new_batch = new_batch+[get_embedding([str(x[0])], embed_dict, prior) for x in this_sequence[:-1]]
        if (len(this_sequence[:-1])< max_length):
            new_batch = new_batch+[[0]*embed_size for x in range((max_length+1 - len(this_sequence)))]

    return np.asarray(new_batch).reshape(len(sequences), max_length, embed_size), np.asarray(targets).reshape(-1,len(ids)), np.reshape(target_labels,(-1,1)), np.asarray(target_indexes).reshape(-1,1), np.asarray(future_context).reshape(-1,embed_future), np.asarray(target_merchs).reshape(-1,1), np.asarray(target_pubs).reshape(-1,1), np.asarray(history).reshape(-1,1), np.asarray(history_no).reshape(-1,1)


# get the correct indexes of non zero paddings.
def get_relevant(rnn_output, indx):
    """
    rnn_output: output from GRU.
    indx: indexes of items to gathered.
    """
    out_size = int(rnn_output.get_shape()[2])
    flat = tf.reshape(rnn_output, [-1, out_size])
    relevant = tf.gather(flat, indx)
    return tf.reshape(relevant,[-1,out_size])    
    

# calculate the length of each sequence before paddding
def lengths(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    used.get_shape()
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length,used # returns the length of each sequence in a batch    

def optimize(data, embed_dictionary, batch_size, num_epochs, max_length, embed_size, lr, TEST_DATA):
    num_batches = int(len(data)//batch_size)
    num_batches_test = int(len(TEST_DATA)//batch_size)
    epoch_acc = []
    epoch_recall, epoch_mrr, epoch_m, epoch_p, epoch_h, epoch_nh, epoch_lens  = [], [], [], [], [], [],[]
    costs = []
    test_counter = 0
    stop_count = 0
    recall_list, mrr_list = [], []
    test_freq = int(num_batches//10)
    with tf.Session() as sess:
        sess.run(init)


        acc_list=[]
        summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())        
        start_epoch = time.time()
        # For each epoch loop over all batches and optimize the cost and produce the test cost
        for epoch in range(num_epochs):
            print("-------Running Epoch:{}-------".format(epoch+1))
            epoch_loss = 0
            np.random.shuffle(data)
            start = time.time()
            freq = int(num_batches//40)
            batch_time = 0
            len_recall = np.zeros((max_length,1))
            len_counts = np.zeros((max_length,1))
            for i in range(num_batches):
                count = 0
                recall = 0

                batch_data = data[i*batch_size:(i+1)*batch_size]


                # Get the batches ready and into the correct form and shape                
                epoch_x, epoch_y, label_ind, out_ind, fut_context, m_ind, p_ind, h_ind, nh_ind = batch_generation(batch_data, embed_dictionary, batch_size, max_length, embed_size, KEY_WORDS,   RANDOM = False)
                feed_dict={seq: epoch_x, targets: epoch_y, train_labels:label_ind, num_samples: n_samples, output_indexes: out_ind, context_future:fut_context, merchant_labels : m_ind, publisher_labels : p_ind, history_labels: h_ind, no_history_labels: nh_ind}
                _, c, acc, this_recall, mrr, m_recall, p_recall, h_recall, nh_recall, r_hits, summary = sess.run([optimizer, loss, accuracy, final_recall, recip_rank, M_final_recall, P_final_recall,  H_final_recall, NH_final_recall, recall_hits,merged_summary_op], feed_dict)              
                r_hits = np.reshape(r_hits, (-1,1))
                #summary_writer.add_summary(summary, epoch * num_batches + i)
                for j in range(max_length):
                    if j in out_ind % 19:
                        len_indx = out_ind%19 == j
                        len_counts[j] += 1
                        len_indx = r_hits[len_indx]
                        len_recall[j] += np.mean(len_indx)
                recall_list.append(this_recall)        
                epoch_loss += c
                mrr_list.append(mrr)
                costs.append(c)
                acc_list.append(acc)
                summary_writer.add_summary(summary, epoch * num_batches + i)
                if (i)% (freq) ==0 or i == (num_batches):
                    saver.save(sess= sess, save_path = save_model) 
                    print("-------Running group:{} out of :{}-------".format(i+1, num_batches))
                    print("Trained {} batches with current batch cost: {} and accuracy: {}, recall {}".format(i+1,c, acc,this_recall))
                    print("Trained {} batches with average accuracy: {}, average cost {}, average recall{}".format(i+1,np.sum(acc_list)/len(acc_list),np.sum(costs)/len(acc_list), np.sum(recall_list)/len(acc_list)))
                    print("Current run time: {} ".format(time.time()-start_epoch))
                    print("Time per {} batches: {}\n ".format(freq, time.time()-batch_time))
                    batch_time = time.time()
                if i% test_freq == 0 or i == num_batches:
                    np.random.shuffle(TEST_DATA)
                    batch_time  =time.time()
                    batch_data = TEST_DATA[:batch_size]
                    epoch_x, epoch_y, label_ind, out_ind, fut_context, m_ind, p_ind, h_ind, nh_ind = batch_generation(batch_data, embed_dictionary, batch_size, max_length, embed_size, KEY_WORDS,   RANDOM = False)
                    feed_dict={seq: epoch_x, targets: epoch_y, train_labels:label_ind, num_samples: n_samples, output_indexes: out_ind, context_future:fut_context, merchant_labels : m_ind, publisher_labels : p_ind, history_labels: h_ind, no_history_labels: nh_ind}
                    acc,  this_recall, mrr, m_recall, p_recall, h_recall, nh_recall = sess.run([  accuracy,final_recall, recip_rank, M_final_recall, P_final_recall, H_final_recall, NH_final_recall ], feed_dict)
                    print("##############################################")
                    print("-------Running Test after:{} out of :{}-------".format(i+1, num_batches))
                    print("Trained {} batches with current batch accuracy: {}, recall {}, mrr {} ".format(i+1, acc,this_recall, mrr))
                    print("Current run time: {} ".format(time.time()-start_epoch))
                    print("Time per batch: {}\n ".format( time.time()-batch_time))
                    print("##############################################\n")
                    test_counter+=1
            recall_by_len_train = np.divide(len_recall, len_counts)
            where_are_NaNs = np.isnan(recall_by_len_train)
            recall_by_len_train[where_are_NaNs] = 0
            print(recall_by_len_train)
            len_recall = np.zeros((max_length,1))
            len_counts = np.zeros((max_length,1))
        
            acc_test,recall_test, R_acc_test, R_recall_test = [], [], [], []
            mrr_test, R_mrr_test = [], []
            mer_recall, pub_recall = [], []
            R_mer_recall, R_pub_recall = [], []
            h_recall = 0
            nh_recall = 0
            R_h_recall = 0
            R_nh_recall = 0
            for i in range(num_batches_test):
                batch_data = TEST_DATA[i*batch_size:(i+1)*batch_size]


                # Get the batches ready and into the correct form and shape                
                epoch_x, epoch_y, label_ind, out_ind, fut_context, m_ind, p_ind, h_ind, nh_ind = batch_generation(batch_data, embed_dictionary, batch_size, max_length, embed_size, KEY_WORDS,   RANDOM = False)
                feed_dict={seq: epoch_x, targets: epoch_y, train_labels:label_ind, num_samples: n_samples, output_indexes: out_ind, context_future:fut_context, merchant_labels : m_ind, publisher_labels : p_ind, history_labels: h_ind, no_history_labels: nh_ind}
                acc,  this_recall, mrr, m_recall, p_recall, h_r, nh_r, r_hits = sess.run([  accuracy,final_recall, recip_rank, M_final_recall, P_final_recall, H_final_recall, NH_final_recall, recall_hits ], feed_dict)
                r_hits = np.reshape(r_hits, (-1,1))
                for j in range(max_length):
                    if j in out_ind % 19:
                        len_indx = out_ind%19 == j
                        len_counts[j] += 1
                        len_indx = r_hits[len_indx]
                        len_recall[j] += np.mean(len_indx)
                        
                acc_test.append(acc)
                recall_test.append(this_recall)
                mrr_test.append(mrr)
                mer_recall.append(m_recall)
                pub_recall.append(p_recall)
                h_recall+= h_r
                nh_recall += nh_r
                
                epoch_x, epoch_y, label_ind, out_ind, fut_context, m_ind, p_ind, h_ind, nh_ind = batch_generation(batch_data, embed_dictionary, batch_size, max_length, embed_size, KEY_WORDS,   RANDOM = True)
                feed_dict={seq: epoch_x, targets: epoch_y, train_labels:label_ind, num_samples: n_samples, output_indexes: out_ind, context_future:fut_context, merchant_labels : m_ind, publisher_labels : p_ind, history_labels: h_ind, no_history_labels: nh_ind}
                R_acc,  R_this_recall, R_mrr, R_m_recall, R_p_recall, R_h_r, R_nh_r =  sess.run([  accuracy,final_recall, recip_rank, M_final_recall, P_final_recall, H_final_recall, NH_final_recall ], feed_dict)
                R_acc_test.append(R_acc)
                R_recall_test.append(R_this_recall)
                R_mrr_test.append(R_mrr)
                R_mer_recall.append(R_m_recall)
                R_pub_recall.append(R_p_recall)
                R_h_recall+= R_h_r
                R_nh_recall += R_nh_r
            recall_by_len_test = np.divide(len_recall, len_counts)
            where_are_NaNs = np.isnan(recall_by_len_test)
            recall_by_len_test[where_are_NaNs] = 0                
            print("Test recall: {}, Test accuracy: {}, merchant recall: {}, publisher recall {}, mrr {}, history {}, no history{}".format(np.mean(recall_test),np.mean(acc_test), np.mean(mer_recall), np.mean(pub_recall), np.mean(mrr_test), h_recall/(i+1), nh_recall/(i+1)))
            print("RANDOM: Test recall: {}, Test accuracy: {},  merchant recall: {}, publisher recall {}, mrr {}, history {}, no history{}".format(np.mean(R_recall_test),np.mean(R_acc_test), np.mean(R_mer_recall), np.mean(R_pub_recall), np.mean(R_mrr_test), R_h_recall/(i+1), R_nh_recall/(i+1)))
            print(recall_by_len_test)
            print("Total time taken for epoch : {:f}".format(time.time()-start_epoch)) 
            epoch_acc.append(np.mean(acc_test))
            epoch_recall.append(np.mean(recall_test)) 
            epoch_mrr.append(np.mean(mrr_test))
            epoch_m.append(np.mean(mer_recall))
            epoch_p.append(np.mean(pub_recall))
            epoch_h.append(np.mean(h_recall))
            epoch_nh.append(np.mean(nh_recall))
            epoch_lens.append(recall_by_len_test)
            if epoch_acc[epoch] - epoch_recall[epoch-1]<0:
                lr = lr/2
                stop_count+=1
                if stop_count == 4:
                    
                    return  epoch_acc, epoch_recall, epoch_mrr, epoch_m, epoch_p, epoch_h, epoch_nh
    return  epoch_acc, epoch_recall, epoch_mrr, epoch_m, epoch_p, epoch_h, epoch_nh

def reommend_pub_merchants(data, embed_dictionary, batch_size, max_length, embed_size, recall_num):   
    num_batches = int(len(data)//batch_size)
    with tf.Session() as sess:
        sess.run(init)
        saver2restore = tf.train.Saver()
        saver2restore.restore(sess = sess, save_path= save_model)
        p_rec_list,m_rec_list = [], []                       
        for i in range(num_batches):
            batch_data = data[i*batch_size:(i+1)*batch_size]
            epoch_x, epoch_y, label_ind, out_ind, fut_context, m_ind, p_ind, h_ind, nh_ind = batch_generation(batch_data, embed_dictionary, batch_size, max_length, embed_size, KEY_WORDS,   RANDOM = False)
            feed_dict={seq: epoch_x, targets: epoch_y, train_labels:label_ind, num_samples: n_samples, output_indexes: out_ind, context_future:fut_context, merchant_labels : m_ind, publisher_labels : p_ind, history_labels: h_ind, no_history_labels: nh_ind}
            p_recs, m_recs = sess.run([  top_n_pubs, top_n_merchants ], feed_dict)
            p_recs = p_recs.reshape(1,-1).tolist()
            m_recs = m_recs.reshape(1,-1).tolist()
            p_rec_list = p_rec_list + [mapping_reverse[item+len(this_merchants)]  for item in p_recs[0]]
            m_rec_list = m_rec_list +[mapping_reverse[item]  for item in m_recs[0]]
        return np.asarray(p_rec_list).reshape(-1, recall_num), np.asarray(m_rec_list).reshape(-1, recall_num)

# read in the dictionary which is the embedding from spark
with open(r"/home/alex/Skimlinks/kw_data/one_week_embeddings.pkl", "rb") as input_file:
    dictionary = pickle.load(input_file)

embedding_list = dictionary.values()
domain_ids = dictionary.keys()
dictionary['0'] = [0.0]*50

with open(r"/home/alex/Skimlinks/kw_data/kw.pkl", "rb") as input_file:
    kw = pickle.load(input_file)

with open(r"/home/alex/Skimlinks/kw_data/merchants.pkl", "rb") as input_file:
    merchants = pickle.load(input_file)

def append_keyword(ids, this_kws, lst_domain_kw):
    for item in set(this_kws):
        lst_domain_kw.append(list((ids,item))) 
    if key_words == []:
        lst_domain_kw.append(list((ids,"")))

merchant_set = set(merchants)

start = time.time()
with open('/home/alex/Skimlinks/kw_data/sessions.txt', 'rt') as f:
    reader= csv.reader(f, delimiter=',')
    data, d_kw = [], []
    count = 0
    for row in reader:
        this_row = literal_eval(','.join(row))
        key_words = []
        session_domains = [x[1] for x in this_row]
        prev_did = str(session_domains[0])
        
        # get sessions with at least 2 different domains
        if len(set(session_domains))>1:
            tmp, dt, ts = [], [], []
            for elm in this_row:
                did = str(elm[1])
                if did == prev_did:
                    key_words.extend(elm[2])
                    dt.append(np.log2(elm[-1]))
                    ts.append(elm[0]/3600 % 24)
                    
                else:
                    if prev_did in merchant_set:
                        m=1
                    else: m = 0
                    tmp.append(list((prev_did,key_words,np.round(np.mean(dt)).tolist(), ts[-1], elm[-2],m)))
                    append_keyword(prev_did, key_words, d_kw)
                        
                    key_words, dt, ts = [], [], []
                    dt.append(np.log2(elm[-1]))
                    key_words.extend(elm[2])
                    ts.append(elm[0]/3600 % 24)
                    prev_did = did
                
                if elm == this_row[-1]: 
                    if prev_did in merchant_set:
                        m=1
                    else: m =0
                    tmp.append(list((prev_did,key_words,np.round(np.mean(dt)).tolist(), ts[-1], elm[-2],m)))
                    append_keyword(prev_did, set(key_words), d_kw)
            data.append(tmp)
        count+=1
        if count % 100000 == 0:
            print("processing row :{}, time taken so far:{}".format(count, time.time() - start))
            
print(time.time()- start)              
# ID, kw, date, dt        
    
df = pd.DataFrame(d_kw)
df.columns = ['domain', 'kw']
gb = df.groupby(['domain'])
domain_kw = {k: set(v) for k,v in gb["kw"]}


size_dt = 20
size_id = 2
size_hour = 24
size_day = 31 
recall_number = 10

all_domains = gb.groups.keys()

# get the domain ids since I forgot to earlier
domains = set(all_domains)
merchant_set = set(merchants)
this_merchants = set.intersection(merchant_set,domains)
pub_ids = domains - this_merchants
this_merchants = list(this_merchants)
pub_ids = list(pub_ids)
pub_ids = [str(x) for x in pub_ids]

av_kw_domain_embed = {}
for elm in domain_kw.keys():
    current_kw = domain_kw[elm]
    current_embed = average_keywords_domains(current_kw, kw)
    av_kw_domain_embed[elm] = current_embed

# get the average merchants
prior = np.reshape(np.mean(np.asarray(merchant_average(this_merchants, dictionary)),axis = 0),(-1,1))

ids = this_merchants+ pub_ids
indx = range(len(this_merchants)+ len(pub_ids))
# Getting a one hot encoding by indexing each id.
mapping = dict(zip(ids,indx))
mapping_reverse = dict(zip(indx, ids))

KEY_WORDS = True
embed_size = 50
embed_future = 300 +size_dt+ size_id+size_hour+ size_day# chosen prior
if KEY_WORDS == True:
    embed_size = 50 + 300 
    
# define parameters
rnn_size = 100 # hidden layer size
output_size = len(ids) # output after rnn
max_length = 19 # max seq length is 20, last is taken as target
learning_rate = 0.0015
batch_size = 128

tf.reset_default_graph()
# placeholders tp store the inputs and labels 
seq = tf.placeholder(tf.float32, [None, max_length ,embed_size],name='seq')

targets = tf.placeholder(tf.float32,shape  = [None,None],name='LabelData')

train_labels = tf.placeholder(tf.int32, shape=[None,1])

output_indexes = tf.placeholder(tf.int32, shape=[None,1], name = 'indexes_data')

num_samples = tf.placeholder(tf.int32, shape = [],name = "Negative_sample_num")

context_future = tf.placeholder(tf.float32,shape  = [None,embed_future],name='future_context')

merchant_labels = tf.placeholder(tf.int32, shape=[None,1])

publisher_labels = tf.placeholder(tf.int32, shape=[None,1])

history_labels = tf.placeholder(tf.int32, shape=[None,1])

no_history_labels = tf.placeholder(tf.int32, shape=[None,1])

variables = {'weights':tf.Variable(tf.random_normal([rnn_size,output_size],stddev= 0.1),name='Weights1'),
             'biases':tf.Variable(tf.random_normal([output_size],stddev = 0.1),name='Bias')}
    
    
# Here the gru cell is defined of specified size
gru_cell = tf.contrib.rnn.GRUCell(rnn_size)

# The ouputs are a tensor of all the ouput states of the pixels
outputs, states = tf.nn.dynamic_rnn(cell = gru_cell, inputs = seq, dtype=tf.float32, sequence_length=lengths(seq)[0],scope = 't')


#last = last_relevant(outputs, length(seq)[0])
last = get_relevant(outputs,output_indexes)


output = tf.matmul(last,variables['weights'])+variables['biases']

top_n, top_n_index =  tf.nn.top_k(output,num_samples+1)

# basically we end up repeating the target score before the softmax across the cols, 
# so that we can use it tp subtract later
cat_idx = tf.concat([tf.reshape(tf.range(0, tf.shape(output)[0]), [-1,1]), train_labels], axis=1, name = "Targets")

target_ind = tf.reshape(tf.gather_nd(output, cat_idx),[-1,1])

# Tensor(matrix) with repeated target values repated for the number of negative samples 
target_ind = tf.tile(target_ind, tf.stack([1, num_samples]))

top_neg_vals, top_neg_indice =  tf.nn.top_k(output,num_samples)

top_5_indx = tf.slice(top_neg_indice,[0,0],[-1,recall_number])
#top_5_indx = tf.nn.top_k(output,recall_number).indices
# bpr max loss

diff =  target_ind - top_neg_vals

activate = tf.sigmoid(diff)

# need to check if the softmax is with resoect to all or just sampled
softmax_scores = tf.nn.softmax(top_neg_vals)

lmbda = 1
regularizer = softmax_scores*tf.square(top_neg_vals)
loss = activate * softmax_scores

with tf.name_scope('bpr_Loss'):
    loss = tf.reduce_mean(-tf.log(tf.reduce_sum(loss,1))+ lmbda* tf.reduce_sum(regularizer,1))
# Xent =  tf.nn.softmax_cross_entropy_with_logits(logits = output, labels = targets)
# with tf.name_scope('bpr_Loss'):
#      loss = tf.reduce_mean(Xent)


optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

with tf.name_scope('Accuracy'):
    correct_label = tf.equal(tf.argmax(output, 1), tf.argmax(targets, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_label, tf.float32))

with tf.name_scope('Mean_Reciprical_rank'):
    ordered_ranks = tf.cast(tf.nn.top_k(output,len(ids)).indices, tf.float32, name = 'take_top')
    mrr_labels = tf.cast(tf.tile(train_labels, [1, len(ids)]),tf.float32)
    ranks = tf.cast(tf.argmax(tf.cast(tf.equal(mrr_labels, ordered_ranks), tf.float32),1) +1, tf.float32)
    recip_rank = tf.reduce_mean(tf.reciprocal(ranks))

    

repeated_labels = tf.tile(train_labels, [1, recall_number])  # Create multiple columns.
correct_recall = tf.equal(repeated_labels, top_5_indx)
correct_recall = tf.cast(correct_recall, tf.float32)
recall_hits = tf.reduce_sum(correct_recall, 1)


with tf.name_scope("Recall"):
    final_recall = tf.reduce_mean(tf.reduce_sum(correct_recall, 1))
    

with tf.name_scope("Merchant_Recall"):
    M_correct_recall = tf.reshape(tf.gather(correct_recall, merchant_labels),[-1, recall_number])
    M_final_recall = tf.reduce_mean(tf.reduce_sum(M_correct_recall, 1))
    

with tf.name_scope("Publisher_Recall"):
    P_correct_recall = tf.reshape(tf.gather(correct_recall, publisher_labels),[-1, recall_number])
    P_final_recall = tf.reduce_mean(tf.reduce_sum(P_correct_recall, 1))

    
with tf.name_scope("History_Recall"):
    H_correct_recall = tf.reshape(tf.gather(correct_recall, history_labels),[-1, recall_number])
    H_final_recall = tf.reduce_mean(tf.reduce_sum(H_correct_recall, 1))

    
with tf.name_scope("NoHistory_Recall"):
    NH_correct_recall = tf.reshape(tf.gather(correct_recall, no_history_labels),[-1, recall_number])
    NH_final_recall = tf.reduce_mean(tf.reduce_sum(NH_correct_recall, 1))
# return merchant recommedations during testing
top_n_merchants = tf.nn.top_k(tf.slice(output,[0,0],[-1,len(this_merchants)]),recall_number).indices
top_n_pubs = tf.nn.top_k(tf.slice(output,[0,len(this_merchants)],[-1,len(pub_ids)]),recall_number).indices


# Create a summary to monitor cost tensor
tf.summary.scalar("loss_pbpr", loss)
tf.summary.scalar("accuracy", accuracy)
tf.summary.scalar("Publisher_Recall", P_final_recall)
tf.summary.scalar("Merchant_Recall", M_final_recall)
tf.summary.scalar("History_Recall", H_final_recall)
tf.summary.scalar("NoHistory_Recall", NH_final_recall)
tf.summary.scalar("Recall", final_recall)
tf.summary.scalar("Mean_Reciprical_rank", recip_rank)

# Merge all summaries into a single op

logs_path = '/tmp/tensorflow_logs/key_words_context'
merged_summary_op = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

# Need to save the model, weights and biases varibles
saver = tf.train.Saver(write_version = tf.train.SaverDef.V2)

# Suggested Directory to use
save_MDir = 'models/'


#create the directory if it does not exist already
if not os.path.exists(save_MDir):
    os.makedirs(save_MDir)

save_model = os.path.join(save_MDir,'model_context_features')

init = tf.global_variables_initializer()
n_samples = len(ids)//3
restoreModel = False
if restoreModel == False:
    accs,recalls, mrrs, r_m, r_p, r_h, r_nh  = optimize(data[:-100000], dictionary, 128, 5, max_length, embed_size, learning_rate, data[-100000:])
else:    
    # name of directory to read model from
    save_MDir = 'models/'
    save_model = os.path.join(save_MDir,'model_context_features') 
    pub_recomendations, merchant_recommendations = reommend_pub_merchants(data[-10000:], dictionary, 128, max_length, embed_size, recall_number)